In [4]:
from collections import defaultdict
import heapq

Definition des variables necessaires

In [5]:
num_rooms_in_trajectory = 4
room_time = 180
transition_time = 30
immersion_groups = []
total_immersion_time = num_rooms_in_trajectory * room_time + (num_rooms_in_trajectory - 1) * transition_time

# Main parameters
room_size = [3, 3]
num_virtual_rooms = room_size[0] * room_size[1]

Definition d'une classe groupe

In [6]:
class Group:
    def __init__(self, group_id, num_people, initial_departure_time, room_time, transition_time, trajectory=[], room_times={}):
        self.group_id = group_id
        self.num_people = num_people
        self.initial_departure_time = initial_departure_time
        self.room_time = room_time
        self.transition_time = transition_time
        self.trajectory = trajectory
        self.room_times = room_times

    def add_room(self, room,):
        self.trajectory.append(room)
        
        # Better to update this directly when adding the room
    def define_room_times(self):
        self.room_times = {room: [] for room in self.trajectory}
        print(self.room_times)
        for i in range(len(self.trajectory)): 
            print(i)
            self.room_times[self.trajectory[i]].append([self.initial_departure_time + i*(self.room_time + self.transition_time), self.initial_departure_time + (i+1)*(self.room_time) + i*self.transition_time])
            print(self.room_times)

    def display_information(self):
        print(f"Group ID: {self.group_id}")
        print(f"Number of people in the group: {self.num_people}")
        print(f"Initial departure time: {self.initial_departure_time}")
        print("Group trajectory:")
        for room in self.trajectory:
            print(f"- {room}")
        for (key, values) in self.room_times.items():
            print((key, values))


In [21]:
group1 = Group(1, 5, 0, room_time, transition_time, room_times={})
group1.trajectory = []
group1.add_room(0)
group1.add_room(8)
group1.add_room(2)
group1.add_room(4)
group1.add_room(4)
group1.define_room_times()
immersion_groups.append(group1)

{0: [], 8: [], 2: [], 4: []}
0
{0: [[0, 180]], 8: [], 2: [], 4: []}
1
{0: [[0, 180]], 8: [[210, 390]], 2: [], 4: []}
2
{0: [[0, 180]], 8: [[210, 390]], 2: [[420, 600]], 4: []}
3
{0: [[0, 180]], 8: [[210, 390]], 2: [[420, 600]], 4: [[630, 810]]}
4
{0: [[0, 180]], 8: [[210, 390]], 2: [[420, 600]], 4: [[630, 810], [840, 1020]]}


In [22]:
group2 = Group(2, 5, 180, room_time, transition_time, room_times={})
group2.trajectory = []
group2.add_room(0)
group2.add_room(3)
group2.add_room(7)
group2.add_room(1)
group2.define_room_times()
immersion_groups.append(group2)

{0: [], 3: [], 7: [], 1: []}
0
{0: [[180, 360]], 3: [], 7: [], 1: []}
1
{0: [[180, 360]], 3: [[390, 570]], 7: [], 1: []}
2
{0: [[180, 360]], 3: [[390, 570]], 7: [[600, 780]], 1: []}
3
{0: [[180, 360]], 3: [[390, 570]], 7: [[600, 780]], 1: [[810, 990]]}


In [23]:
group3 = Group(3, 6, 180, room_time, transition_time, room_times={})
group3.trajectory = []
group3.add_room(0)
group3.add_room(1)
group3.add_room(4)
group3.add_room(2)
group3.define_room_times()
immersion_groups.append(group3)

{0: [], 1: [], 4: [], 2: []}
0
{0: [[180, 360]], 1: [], 4: [], 2: []}
1
{0: [[180, 360]], 1: [[390, 570]], 4: [], 2: []}
2
{0: [[180, 360]], 1: [[390, 570]], 4: [[600, 780]], 2: []}
3
{0: [[180, 360]], 1: [[390, 570]], 4: [[600, 780]], 2: [[810, 990]]}


In [24]:
check_group = 2
num_people_first_group = immersion_groups[check_group].num_people
print(num_people_first_group)
print(immersion_groups[check_group].trajectory)
print(immersion_groups[check_group].room_times)


6
[0, 1, 4, 2]
{0: [[180, 360]], 1: [[390, 570]], 4: [[600, 780]], 2: [[810, 990]]}


Definition des fonctions de populations qui renvoie a un temps t donné, un array qui represente le nombre de personnes presentes dans chacune des 9 salles

In [7]:
def population(t, immersion_groups):
    # It is zero everywhere at the beginning
    room_populations = [0] * num_virtual_rooms

    # Traverse the intervals to find the active groups
    for group in immersion_groups:
        room_times = group.room_times
        for key, values in room_times.items():
            for value in values:
                if value[0] <= t <= value[1]:
                    room_populations[key] += group.num_people
    
    return room_populations

def population_in_point(t, immersion_groups, point):
    # It is zero everywhere at the beginning
    room_population = 0

    # Traverse the intervals to find the active groups
    for group in immersion_groups:
        room_times = group.room_times
        for key, values in room_times.items():
            for value in values:
                if key == point and (value[0] <= t <= value[1]):
                    room_population += group.num_people
    
    return room_population


In [8]:
print(population(850, immersion_groups))
print(population_in_point(850, immersion_groups, 2))

[0, 0, 0, 0, 0, 0, 0, 0, 0]
0


Definition de l'integrale de la population qui sont les poids que l'on veut utiliser pour notre graphe pondéré

In [9]:
def population_integral(t_start, t_end, immersion_groups):
    # Number of slices for numerical approximation
    num_slices = 100
    
    # Width of each slice
    h = (t_end - t_start) / num_slices
    
    # Initialization of the integral array
    integral_array = [0] * num_virtual_rooms
    
    # Calculating the integral for each element of the population array
    for i in range(num_slices + 1):
        t = t_start + i * h
        populations = population(t, immersion_groups)
        for j, room_population in enumerate(populations):
            if i == 0 or i == num_slices:
                integral_array[j] += room_population * h / 2
            else:
                integral_array[j] += room_population * h
    
    return integral_array

def population_integral_in_point(t_start, t_end, immersion_groups, point):
    # Number of slices for numerical approximation
    num_slices = 100
    
    # Width of each slice
    h = (t_end - t_start) / num_slices
    
    # Initialization of the integral array
    integral_array = 0
    
    # Calculating the integral for each element of the population array
    for i in range(num_slices + 1):
        t = t_start + i * h
        population = population_in_point(t, immersion_groups,point)
        if i == 0 or i == num_slices:
            integral_array += population * h / 2
        else:
            integral_array += population * h
    
    return integral_array


In [10]:
print(population_integral(160, 250, immersion_groups))
print(population_integral_in_point(160, 250, immersion_groups, 5))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0


In [11]:
import numpy as np
from itertools import permutations

def calculate_transition_cost(t, point_A, point_B, debug = False):
    cost = population_integral_in_point(t, t + room_time, immersion_groups, point_B)
    if debug:
        print('Going from point '+ str(point_A) +' to point ' + str(point_B))
        print('the time is ' + str(t))
        print('The transition vector is ' + str(costs_vector))
        print('it costs '+ str(costs_vector[point_B]) + ' for the transition.\n')
    return cost

# Function to calculate the total cost of a room assignment
def calculate_total_cost(t, assignment, routine, debug=False):
    total_cost = 0
    for i in range(len(routine)-1):
        total_cost += calculate_transition_cost(t + room_time * (i), assignment[routine[i]], assignment[routine[i + 1]])
        if debug:
            print('Going from '+ routine[i] +' to ' + routine[i+1])
            calculate_transition_cost(t + room_time * (i), assignment[routine[i]], assignment[routine[i + 1]], debug)
    return total_cost

def get_dictionary(routine, permutation):
    assignment_counter = 0
    assignment_dict = dict.fromkeys(routine)
    for keys in assignment_dict:
        if keys == 'intro':
            assignment_dict[keys] = 0
        else:
            assignment_dict[keys] = permutation[assignment_counter]
            assignment_counter +=1
    return assignment_dict

# Find the room assignment that minimizes the total cost

def get_rooms_assignment(start_time, routine, debug = False):
    min_cost = float('inf')
    best_assignment = None
    nodes = range(1, num_virtual_rooms)
    room_assignments = permutations(nodes, len(dict.fromkeys(routine[1:])))

    for assignment in room_assignments:
        assignment_dict = get_dictionary(routine, assignment)
        total_cost = calculate_total_cost(start_time, assignment_dict, routine)
        if total_cost < min_cost:
            min_cost = total_cost
            best_assignment = assignment
        if min_cost == 0:
            break

    best_assignment_dict = get_dictionary(routine, best_assignment)
    if debug:
        calculate_total_cost(start_time, best_assignment_dict, routine, debug = True)
    
    return best_assignment_dict, min_cost



In [12]:

def add_group(start_time, num_persons, routine):
    best_assignment, min_cost = get_rooms_assignment(start_time, routine)
    # Print the best room assignment and its minimum total cost
    print("\nBest room assignment:", best_assignment)
    print("Minimum total cost:", min_cost)
    
    group = Group(len(immersion_groups), num_persons, start_time, room_time, transition_time, room_times={})
    group.trajectory = []
    for i in routine:
        group.add_room(best_assignment[i])
    group.define_room_times()
    immersion_groups.append(group)

In [13]:
import random
start_times = [0, 200, 380, 500, 700]
routine = ['intro', 'hall', 'garden', 'hall', 'room']

for i in start_times:
    add_group(i, random.randint(4, 7), routine)


Best room assignment: {'intro': 0, 'hall': 1, 'garden': 2, 'room': 3}
Minimum total cost: 0.0
{0: [], 1: [], 2: [], 3: []}
0
{0: [[0, 180]], 1: [], 2: [], 3: []}
1
{0: [[0, 180]], 1: [[210, 390]], 2: [], 3: []}
2
{0: [[0, 180]], 1: [[210, 390]], 2: [[420, 600]], 3: []}
3
{0: [[0, 180]], 1: [[210, 390], [630, 810]], 2: [[420, 600]], 3: []}
4
{0: [[0, 180]], 1: [[210, 390], [630, 810]], 2: [[420, 600]], 3: [[840, 1020]]}

Best room assignment: {'intro': 0, 'hall': 3, 'garden': 4, 'room': 2}
Minimum total cost: 0.0
{0: [], 3: [], 4: [], 2: []}
0
{0: [[200, 380]], 3: [], 4: [], 2: []}
1
{0: [[200, 380]], 3: [[410, 590]], 4: [], 2: []}
2
{0: [[200, 380]], 3: [[410, 590]], 4: [[620, 800]], 2: []}
3
{0: [[200, 380]], 3: [[410, 590], [830, 1010]], 4: [[620, 800]], 2: []}
4
{0: [[200, 380]], 3: [[410, 590], [830, 1010]], 4: [[620, 800]], 2: [[1040, 1220]]}

Best room assignment: {'intro': 0, 'hall': 5, 'garden': 6, 'room': 1}
Minimum total cost: 0.0
{0: [], 5: [], 6: [], 1: []}
0
{0: [[380, 56

In [20]:

immersion_groups.clear()

import random

def generate_random_vector(N, min_val, max_val, separation):
    vector = [random.randint(min_val, max_val)]
    for _ in range(N - 1):
        new_val = vector[-1] + separation + random.randint(min_val, max_val)
        vector.append(new_val)
    return vector

N = 10  # Number of entries in the vector

start_times = generate_random_vector(N, 0, 100, room_time) #intro time is room_time
routine = ['intro', 'hall', 'garden', 'hall', 'room']

for i in start_times:
    add_group(i, random.randint(4, 7), routine)



Best room assignment: {'intro': 0, 'hall': 1, 'garden': 2, 'room': 3}
Minimum total cost: 0.0
{0: [], 1: [], 2: [], 3: []}
0
{0: [[18, 198]], 1: [], 2: [], 3: []}
1
{0: [[18, 198]], 1: [[228, 408]], 2: [], 3: []}
2
{0: [[18, 198]], 1: [[228, 408]], 2: [[438, 618]], 3: []}
3
{0: [[18, 198]], 1: [[228, 408], [648, 828]], 2: [[438, 618]], 3: []}
4
{0: [[18, 198]], 1: [[228, 408], [648, 828]], 2: [[438, 618]], 3: [[858, 1038]]}

Best room assignment: {'intro': 0, 'hall': 3, 'garden': 1, 'room': 2}
Minimum total cost: 0.0
{0: [], 3: [], 1: [], 2: []}
0
{0: [[274, 454]], 3: [], 1: [], 2: []}
1
{0: [[274, 454]], 3: [[484, 664]], 1: [], 2: []}
2
{0: [[274, 454]], 3: [[484, 664]], 1: [[694, 874]], 2: []}
3
{0: [[274, 454]], 3: [[484, 664], [904, 1084]], 1: [[694, 874]], 2: []}
4
{0: [[274, 454]], 3: [[484, 664], [904, 1084]], 1: [[694, 874]], 2: [[1114, 1294]]}

Best room assignment: {'intro': 0, 'hall': 4, 'garden': 2, 'room': 1}
Minimum total cost: 0.0
{0: [], 4: [], 2: [], 1: []}
0
{0: [[54